In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
from skimage import color

DATADIR = "data/"
fnames = os.listdir(DATADIR)
fnames = [os.path.join(DATADIR, i) for i in fnames]
print(fnames)

In [ ]:
cam1_1 = scipy.io.loadmat(fnames[0])
print(cam1_1.keys())
cam1_1 = cam1_1['vidFrames1_1']

In [ ]:
print(cam1_1.shape)
print(cam1_1[:,:,:,0].shape)
y, x, colors, samples = cam1_1.shape

In [ ]:
y, x, _, samples = cam1_1.shape
image = cam1_1[..., 0]
   
f = plt.figure(1)
p = f.add_subplot(111)
image = p.imshow(image, cmap="gray")       